In [7]:
%matplotlib inline

## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

##
# IMPORTS
##
import nibabel as nib
import dipy.io
#import scipy.io # for writing the .mat files
import numpy as np

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [8]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [9]:
root_dir='/scratch/TESTS/fix_HCP_dwi'

DWI_nifti_fn = os.path.join(root_dir,'input_sample/100307/nifti/data.nii.gz')
bvals_fn = os.path.join(root_dir,'input_sample/100307/nifti/bvals')
bvecs_fn = os.path.join(root_dir,'input_sample/100307/nifti/bvecs')

#input_nrrd_bvecs_fn = os.path.join(root_dir,'input_sample/100307/nrrd/bvecs')
DWI_nrrd_fn = os.path.join(root_dir,'input_sample/100307/nrrd/data.nrrd')
DWI_nrrd_corrected_fn = os.path.join(root_dir,'input_sample/100307/nrrd/data_corrected.nrrd')
DWI_nrrd_corrected_sshell_fn = os.path.join(root_dir,'input_sample/100307/nrrd/data_corrected_sshell.nrrd')

In [3]:
'''
# NOTE
# dwiConvert reads the bvecs in a row-wise, but the gradient directions are saved column-wise,
# so this step is necessary to reorder the bvectors in a proper format for input of dwiconvert
#
from dipy.io import read_bvals_bvecs
bvals, bvecs = read_bvals_bvecs(bvals_fn, bvecs_fn)
np.savetxt(input_nrrd_bvecs_fn,bvecs)
'''

'\n# NOTE\n# dwiConvert reads the bvecs in a row-wise, but the gradient directions are saved column-wise,\n# so this step is necessary to reorder the bvectors in a proper format for input of dwiconvert\n#\nfrom dipy.io import read_bvals_bvecs\nbvals, bvecs = read_bvals_bvecs(bvals_fn, bvecs_fn)\nnp.savetxt(input_nrrd_bvecs_fn,bvecs)\n'

In [4]:
#run DWIConvert to convert nifti to nrrd
dwiConvert = DWIConvert()
dwiConvert.inputs.inputVolume = DWI_nifti_fn
dwiConvert.inputs.conversionMode = 'FSLToNrrd'
dwiConvert.inputs.inputBVectors = bvecs_fn
dwiConvert.inputs.inputBValues = bvals_fn
dwiConvert.inputs.allowLossyConversion = True # input data is float
dwiConvert.inputs.transposeInputBVectors = True # bvecs are saved column-wise
dwiConvert.inputs.outputVolume = DWI_nrrd_fn
print(dwiConvert.cmdline)
dwiConvert.run()

INFO:interface:stdout 2016-06-16T19:20:51.752108:Spacing :[1.25, 1.25, 1.25, 1]
INFO:interface:stdout 2016-06-16T19:20:55.522311:Number of components in converted Nrrd volume: 288
INFO:interface:Could not get resources used by process. Error: (pid=17254)


 DWIConvert  --allowLossyConversion  --conversionMode FSLToNrrd --inputBValues /scratch/TESTS/fix_HCP_dwi/input_sample/100307/nifti/bvals --inputBVectors /scratch/TESTS/fix_HCP_dwi/input_sample/100307/nifti/bvecs --inputVolume /scratch/TESTS/fix_HCP_dwi/input_sample/100307/nifti/data.nii.gz --outputVolume /scratch/TESTS/fix_HCP_dwi/input_sample/100307/nrrd/data.nrrd --transposeInputBVectors 


In [10]:
# 
# READ/WRITE DWI scan using pynrrd
#
import nrrd
from math import sqrt
from collections import OrderedDict

class nrrdDWIHeader:
    """A helper class for manipulating header information
    from a nrrd DWI data set into a nibabel compliant
    format"""
    def __init__(self,pynrrdDataModel):
        self.modality=None                      #Matches NRRD File
        self.global_BValue=None                 #Matches NRRD File
        self.gradientUnormalizedVectors=[None]  #Matches NRRD File

        self.gradientIndex=-1
        self.gradientBValues=[None]             #Computed
        self.gradientVectors=[None]             #Computed
        self._ConvertNrrdToNibabelDWIDataModel(pynrrdDataModel)

    def Print(self):
        print("global_BValue {0}".format(self.global_BValue) )
        print("modality {0}".format(self.modality) )
        print("gradientBValues {0}".format(self.gradientBValues) )
        print("gradientDirections {0}".format(self.gradientUnormalizedVectors) )

    def _getGradientStorageIndex(self,pynrrdDataModel):
        """The 4D index that has separate gradients.
        all other directions are the spatial dimensions."""
        gradient_index=-1;
        centerings=pynrrdDataModel['centerings']
        numdwidims=len(centerings)
        for test_index in range(0,numdwidims):
            if centerings[test_index] not in ['cell']:
                gradient_index=test_index
        self.gradientIndex = gradient_index

    def _ExtractGlobalBValue(self, pyNrrdKVUnknownMetaData):
        globalBValueString=pyNrrdKVUnknownMetaData.get(u'DWMRI_b-value','0').lstrip().rstrip()
        self.global_BValue=float( globalBValueString )

    def _ExtractUnormalizedBValues(self,pyNrrdKVUnknownMetaData,pynrrdDataModel):
        """Unnormalized data values from the nrrd files, where the
        magnitude is reflective of the scale relative to the global_BValue"""
        self._getGradientStorageIndex(pynrrdDataModel)
        numGradients = pynrrdDataModel[u'sizes'][self.gradientIndex]
        self.gradientUnormalizedVectors= np.array( [ [ None, None, None ] for x in range(0,numGradients) ] )
        gvec_Fields=pyNrrdKVUnknownMetaData.copy() #Clone so we can remove items
        for k,v in gvec_Fields.iteritems():
            if k.startswith("DWMRI_gradient_"):
                index=int(k.split('_')[2])
                self.gradientUnormalizedVectors[index,:]=np.array([ float(x) for x in v.rstrip().lstrip().split() ],
                                                    copy=True, order='C', ndmin=1)
                pyNrrdKVUnknownMetaData.pop(k)


    def _ExtractDWIModality(self,pyNrrdKVUnknownMetaData):
        self.modality=pyNrrdKVUnknownMetaData.get("modality","UNKNOWN").lstrip().rstrip()

    def _ComputeNormalizedBvecBVals(self,pyNrrdKVUnknownMetaData,pynrrdDataModel):
        self._ExtractGlobalBValue(pyNrrdKVUnknownMetaData)
        self._ExtractUnormalizedBValues(pyNrrdKVUnknownMetaData,pynrrdDataModel)
        self._ComputeNormalizedGradientsAndBValues()

    def _ComputeNormalizedGradientsAndBValues(self):
        """Following conventions of NRRD format, expand
        to normalized vectors and estimate BValues
        :return: void
        """
        numGradients=len( self.gradientUnormalizedVectors )

        self.gradientVectors = np.array(self.gradientUnormalizedVectors)
        self.gradientBValues = np.array( [ self.global_BValue ] * numGradients )
        for index in range(0,numGradients):
            gv = self.gradientUnormalizedVectors[index]
            norm=np.linalg.norm(gv)
            if norm < 1e-2:
                self.gradientVectors[index] = gv * 0.0
                self.gradientBValues[index] = 0.0
            elif ( abs( 1.0-norm ) > 1e-4 ): # Avoid rescaling if norm is almost one
                self.gradientVectors[index] = gv/norm
                b_i = self.global_BValue * (norm**2) # norm = sqrt(b_i/b_max)
                self.gradientBValues[index] = float("{0:.1f}".format(b_i))  

    def _ConvertNrrdToNibabelDWIDataModel(self, pynrrdDataModel):
        pyNrrdKVUnknownMetaData=pynrrdDataModel['keyvaluepairs']
        self._ExtractDWIModality(pyNrrdKVUnknownMetaData)
        self._ExtractGlobalBValue(pyNrrdKVUnknownMetaData)
        self._ComputeNormalizedBvecBVals(pyNrrdKVUnknownMetaData,pynrrdDataModel)

def ReadNAMICDWIFromNrrd(filename):
    nrrd_dwi_data,nrrd_dwi_header=nrrd.read(filename)
    nibabelDataModelDWI=nrrdDWIHeader(nrrd_dwi_header)
    nrrd_dwi_bvec=nibabelDataModelDWI.gradientVectors
    nrrd_dwi_bval=nibabelDataModelDWI.gradientBValues
    gradient_index=nibabelDataModelDWI.gradientIndex
    return (nrrd_dwi_data, nrrd_dwi_header, nrrd_dwi_bvec, nrrd_dwi_bval, gradient_index)

def WriteNAMICDWIToNrrd(filename, data, bvecs, bvals, options=None):
    """
    :param filename: The filename to write to disk
    :param data: The numpy 4d file to be written
    :param bvecs: The bvecs values to be written
    :param bvals: The bvals to be written
    :param options: Optional parameters to be written to the nrrd header
    :return:
    """
    keyvaluePairDict=OrderedDict()
    if options is not None:
        keyvaluePairDict=OrderedDict( options.get( 'keyvaluepairs',OrderedDict() ))

    ## First remove all existing bval/bvec fields from dictionary
    keyvaluePairDict.pop('modality',None)

    for k,v in keyvaluePairDict.iteritems():
        if k.startswith('DWMRI_'):
            keyvaluePairDict.pop(k, None)

    keyvaluePairDict[u'modality']=u'DWMRI'
    maxBvalue = max(bvals)
    keybval=u'DWMRI_b-value'
    keyvaluePairDict[keybval]=maxBvalue

    numGradients = len(bvecs)
    for index in range(0,numGradients):
        this_scale = sqrt(bvals[index] / maxBvalue)
        this_vec = [ x * this_scale for x in bvecs[index]]
        #print(bvecs[index],this_vec)

        keyvec=u'DWMRI_gradient_{:04d}'.format(index)
        # convert gradient vector value to string (only for consistency with input)
        vec_string= lambda this_vec: u''.join(nrrd._convert_to_reproducible_floatingpoint(x)+'  ' for x in this_vec).rstrip()
        keyvaluePairDict[keyvec]=vec_string(this_vec)
    options['keyvaluepairs']=keyvaluePairDict
    options['encoding']='gzip' # Always use gzip compression for DWI data
    nrrd.write(filename,data,options)

In [11]:
#
nrrd_data,dwi_nrrd_header,nrrd_bvecs,nrrd_bvals,gradient_index = ReadNAMICDWIFromNrrd(DWI_nrrd_fn)
#
# Correct direction cosign so the output data is loaded with correct alignment in Slicer
dwi_nrrd_header[u'space directions'][2][1] = str(-float(dwi_nrrd_header[u'space directions'][2][1]))
#
# Adjust gradient directions based on new correction_matrix
correction_matrix = np.diag([1,-1,1])
nrrd_bvecs = (np.asmatrix(nrrd_bvecs) * correction_matrix).view(type=np.ndarray)

In [12]:
#print(nrrd_bvals)

In [13]:
#print(type(nrrd_bvecs))
#print(nrrd_bvecs)

In [14]:
# Change multi shell data to single shell
useSingleShell = True

def ExtractSingleShell(nrrd_data,nrrd_bvecs,nrrd_bvals,gradient_index):   
    #mask = nrrd_bvals<1010 # corresponds to b=1000
    mask = (nrrd_bvals==5) + (abs(nrrd_bvals/2000-1) < 1e-2) # corresponds to b=2000
    #mask = (nrrd_bvals==5) + (abs(nrrd_bvals/3000-1) < 1e-2) # corresponds to b=3000
    nrrd_bvals = nrrd_bvals[mask]
    nrrd_bvecs = nrrd_bvecs[mask]
    remove_indices = np.where(np.invert(mask))
    nrrd_data = np.delete( nrrd_data, remove_indices, gradient_index)
    return nrrd_data, nrrd_bvecs, nrrd_bvals

if useSingleShell:
    nrrd_data,nrrd_bvecs,nrrd_bvals = ExtractSingleShell(nrrd_data,nrrd_bvecs,nrrd_bvals,gradient_index)

In [15]:
print(len(nrrd_bvals))

108


In [16]:
#print(nrrd_bvecs)

In [17]:
if useSingleShell:
    dwi_file = DWI_nrrd_corrected_sshell_fn
else:
    dwi_file = DWI_nrrd_corrected_fn

In [18]:
# Write output the output DWI nrrd file
WriteNAMICDWIToNrrd(dwi_file,nrrd_data,nrrd_bvecs,nrrd_bvals,dwi_nrrd_header)

In [ ]:
'''
#
# I tried to fix the "space direction" using a correction matrix
#
print(dwi_nrrd_header[u'space directions'])
dwi_nrrd_header[u'space directions'].remove(u'none')
space_direction = np.array([float(x) for x in np.array(dwi_nrrd_header[u'space directions']).ravel()]).reshape(3,3)
correction_matrix = np.diag([1,-1,1])
space_direction = space_direction*correction_matrix
sd_list = str(space_direction.tolist())
dwi_nrrd_header[u'space directions'] = [u'none',sd_list]
print(dwi_nrrd_header[u'space directions'])
'''

In [ ]:
'''
# Test
# Create DTI images from input image and aligned image to check whether the gradient directions are correct in
# the estimated DTI of the aligned image
#
dti_fn = os.path.join(root_dir,'input_sample/100307/nrrd/dti.nrrd')
dti_aligned_fn = os.path.join(root_dir,'input_sample/100307/nrrd/dti_corrected.nrrd')

DTIEstim=dtiestim()
DTIEstim.inputs.method = 'wls'
DTIEstim.inputs.threshold = 0
#DTIEstim.inputs.correctionType = 'nearest'
DTIEstim.inputs.tensor_output = dti_fn
DTIEstim.inputs.dwi_image = DWI_nrrd_fn
print(DTIEstim.cmdline)
DTIEstim.run()

DTIEstim=dtiestim()
DTIEstim.inputs.method = 'wls'
DTIEstim.inputs.threshold = 0
#DTIEstim.inputs.correctionType = 'nearest'
DTIEstim.inputs.tensor_output = dti_aligned_fn
DTIEstim.inputs.dwi_image = DWI_nrrd_aligned_fn
print(DTIEstim.cmdline)
DTIEstim.run()

#
# I verified that gradient directions in the output DTI images are biologically correct.
#
'''

In [ ]:
if useSingleShell:
    ukfTracks = os.path.join(root_dir,'input_sample/100307/nrrd/ukfTracts_sshell.vtp')
else:
    ukfTracks = os.path.join(root_dir,'input_sample/100307/nrrd/ukfTracts_mshell.vtp')

In [ ]:
# This section has two purposes:
# 1) UKFTractography only reads nrrd files (nifti should be converted to nrrd)
# 2) UKF implementation only accepts masks of type 'signed char', 'unsigned char', 'short', and 'unsigned short';
#    therefore, input mask needs to be cast to sitkUInt16 from sitkFloat32
#
brainMask_nifti_fn = os.path.join(root_dir,'input_sample/100307/nifti/nodif_brain_mask.nii.gz')
brainMask_nrrd_fn = os.path.join(root_dir,'input_sample/100307/nrrd/nodif_brain_mask.nrrd')

bmask = sitk.ReadImage(brainMask_nifti_fn)
bmask2 = sitk.Cast(bmask, sitk.sitkUInt16)
sitk.WriteImage(bmask2,brainMask_nrrd_fn)

In [ ]:
# Now run UKFTractorgraphy
'''
UKF = UKFTractography()
UKF.inputs.tracts = ukfTracks
UKF.inputs.numTensor = '2'
UKF.inputs.freeWater = True ## default False
UKF.inputs.minFA = 0.06
UKF.inputs.minGA = 0.06
UKF.inputs.seedFALimit = 0.06
UKF.inputs.Ql = 70
UKF.inputs.recordLength = 2
UKF.inputs.recordTensors = True
UKF.inputs.recordFreeWater = True
UKF.inputs.recordFA = True
UKF.inputs.recordTrace = True
UKF.inputs.dwiFile = dwi_file
UKF.inputs.maskFile = brainMask_nrrd_fn
UKF.inputs.seedsPerVoxel = 2 #int(round((1.25**3)*10/8))
UKF.run()
'''
# light parameters set
UKF = UKFTractography()
UKF.inputs.tracts = ukfTracks
UKF.inputs.numTensor = '2'
UKF.inputs.freeWater = True ## default False
UKF.inputs.minFA = 0.15
UKF.inputs.minGA = 0.1
UKF.inputs.seedFALimit = 0.06
UKF.inputs.Ql = 70
UKF.inputs.recordLength = 2
UKF.inputs.recordTensors = False
UKF.inputs.recordFreeWater = False
UKF.inputs.recordFA = False
UKF.inputs.recordTrace = False
UKF.inputs.dwiFile = dwi_file
UKF.inputs.maskFile = brainMask_nrrd_fn
UKF.inputs.seedsPerVoxel = 1
print(UKF.cmdline)
#UKF.run()

In [ ]:
g0 = np.array([0.0361091986208199,  -0.048164634882149,  0.0367616389697319])
print(np.linalg.norm(g0))

In [ ]:
g0 = np.array([0.020864963687692,  -0.0278309515644456,  0.0212419630315818])
print(np.linalg.norm(g0))

In [ ]:
gv_nifti = np.array([5.44720000e-02,  -3.93730000e-01,   9.17611000e-01])
print(np.linalg.norm(gv_nifti))

In [ ]:
b_max = 3010.0
b = 3005.0
gv_nrrd = np.array([sqrt(b/b_max)*x for x in gv_nifti])
print(gv_nrrd)
print(np.linalg.norm(gv_nrrd))

In [ ]:
gv_nrrd = np.array([0.054427, -0.393403, 0.916849])
print(1-np.linalg.norm(gv_nrrd))

In [ ]:
np.linalg.norm(gv_nrrd)**2 * b_max

In [ ]:
gv_nifti_recovered = np.array([x/np.linalg.norm(gv_nrrd) for x in gv_nrrd])
print(gv_nifti_recovered)

In [ ]:
0.054427 -0.393403 0.916849

In [ ]:
bool(1-(3005.0/3010)>1e-3)